In [10]:
# ============================================================================
# This code is part of Assignment 3 of CS561 (Executive M-Tech -AI Assignment-3)
# Submitted by:
#        IITP001300: Sukhvinder Singh  (email id: sukhvinder.malik13@gmail.com)
#        IITP001316: Manjit Singh Duhan (email id: duhan.manjit@gmail.com)
#        IITP001508: Atul Singh (email id: atulsingh.xcvi@gmail.com)
#============================================================================

In [1]:
import sys
import numpy as np
import heapq
from datetime import datetime

In [2]:
class node_state():
    def __init__(self, matrix, parent=None, move=None, goal=None):
        self.matrix      = np.array(matrix)
        self.parent      = parent           # The parent to this state. The start state will not have any parrent
        self.goal        = goal
        self.move        = move             # What move (Left/Right/Up/Down) has led to this state
        self.max_row     = len(matrix)      # Max rows in the matrix
        self.max_column  = len(matrix[0])   # Max columns in the matrix
        self.hash        = self.get_hash()  # Has of the matrix. Will be used while comparing states.
        self.depth       = self.set_depth() # The depth of this state from the state_state
        self.actual_cost = self.set_actual_cost()  # g(n) i.e. actual cost of this state from the start state
        
        # initializing with 0. It's responsibility of caller to set the cost based on desired heuristic function before using this
        self.cost = 0 
    
    def shape(self):
        return self.max_row, self.max_column
        
    def get_value(self, i, j):
        ''' Get the value from mattrix at [i][j] location '''
        return self.matrix[i, j]
    
    def get_i_j_of(self, val):
        '''Get the i & j of the value in the matrix '''
        return np.where(self.matrix == val)
    
    def get_copy(self):
        ''' Returns the copy of current matrix '''
        return self.matrix.copy()
    
    def set_depth(self):
        ''' Set the depth of this state. The depth will be parent's depth + 1 '''
        if self.parent is not None:
            return self.parent.get_depth() + 1
        else:
            return 0

    def set_actual_cost(self):
        ''' Set the cost i.e. g(n) of this state from the start state.
            As each step is having cost of 1 so we are fixing to to parent cost + 1'''
        if self.parent is not None:
            return self.parent.actual_cost + 1
        else:
            return 0
    
    def get_cost(self):
        return self.cost
        
    def get_depth(self):
        ''' Get function to get the depth of the state '''
        return self.depth

    def __lt__(self, other):
        return self.cost < other.cost
    
    def get_hash(self):
        '''
        Let us create the hash of the matrix so that the we will add the hash of explored state
        in the explored list. Before processing any state, we will check if that state us already
        explored or not.
        To calculate the hash, we are makeing int of the matrix as below
        [[123]
         [456]
         [780]] = 1234567890
        So, every matrix will have a unique hash value
        '''
        multiplier = 100000000
        value = 0
        for i in range(self.max_row):
            for j in range (self.max_column):
                value += self.matrix[i, j] * multiplier
                multiplier //= 10
        return value
    
    def print(self):
        ''' print the matrix '''
        for i in range (self.max_row):
            for j in range (self.max_column):
                val = self.matrix[i, j]
                if val == 0:
                    print("B", end=" ")
                else:
                    print(val, end=" ")
            print(" ")
    
    def get_Heuristics_1(self):
        return 0
    
    def get_Heuristics_2(self):
        cost = 0
        for i in range(self.max_row):
            for j in range(self.max_column):
                #if the current tile is displaced then increment the value
                if self.get_value(i, j) != self.goal.get_value(i, j):
                    cost += 1

        return cost

    
    def get_Heuristics_3(self):
        cost = 0

        for i in range(self.max_row):
            for j in range(self.max_column):
                #Get the value of tile at i, j
                val = self.get_value(i, j)
                
                # Get the value of i, j for the value in goal state
                goal_i, goal_j = self.goal.get_i_j_of(val)
                
                #Manhattan distance = |current_row - goal_row| + |current_column - goal_column|
                distance = abs(i - goal_i) + abs(j - goal_j)
                
                # sum up the Manhattan distance
                cost += distance
        return int(cost)
    
    def get_Heuristics_4(self):
        cost = 0

        for i in range(self.max_row):
            for j in range(self.max_column):
                #Get the value of tile at i, j
                val = self.get_value(i, j)
                
                # Get the value of i, j for the value in goal state
                goal_i, goal_j = self.goal.get_i_j_of(val)
                
                #Euclidean distance = sqrt((current_row - goal_row)^2 + (current_column - goal_column)2)
                distance = np.sqrt((i - goal_i)**2 + (j - goal_j)**2)
                
                # sum up the Euclidean distance
                cost += distance
        return int(cost)
    
    def get_heuristic_cost(self, use_heuristic_function):
        
        if(use_heuristic_function == 1):
            return self.get_Heuristics_1()
        elif(use_heuristic_function == 2):
            return self.get_Heuristics_2()
        elif(use_heuristic_function == 3):
            return self.get_Heuristics_3()
        elif(use_heuristic_function == 4):
            return self.get_Heuristics_4()
        else:
            raise Exception("This is not a supported heuristic function type")
            return 0
        
    def set_cost(self, heuristic_function):
        a = self.actual_cost
        b = self.get_heuristic_cost(heuristic_function)
        self.cost = a + b


In [3]:
class Queue():
    def __init__(self):
        self.list = []
        self.max_size = 0

    def current_size(self):
        return len(self.list)
    
    def clear(self):
        self.list.clear()
    
    def max_grown_size(self):
        return self.max_size
    
    def empty(self):
        return self.current_size() == 0

    def add(self, item):
        self.list.append(item)
        self.max_size = max(self.max_size, self.current_size())   

    def remove(self):
        if self.current_size() == 0:
            raise Exception("Queue is empty")
        else:
            return self.list.pop(0)


class Priority_Queue(Queue):

    def add(self, item):
        heapq.heappush(self.list, (item.get_cost(), item))
        self.max_size = max(self.max_size, self.current_size())

    def remove(self):
        if self.current_size() == 0:
            raise Exception("Priority queue is empty")
        cost, item = heapq.heappop(self.list)
        return item

In [4]:
class puzzle():
    def __init__(self, start, goal):
        # Check if the nput Start state and Goal state are valid or not.
        self.validate_input_matrix(start)
        self.validate_input_matrix(goal)
        
        self.goal_state    = node_state(goal)       
        self.start_state   = node_state(start, goal=self.goal_state)
        
        self.max_row, self.max_column = start.shape
        self.close_list = []   # This list will contain the states that are explored at any moment
        self.solution_steps = []  # This list is used to fill while doing the back-tracing
        self.start_time     = datetime.now()
        self.end_time       = datetime.now()
        self.optimal_path_cost = 0    # sum of g(n) of all states
        self.total_of_fn = 0          # sum of f(n) of all states
        self.number_of_states_to_optimal_path = 0
        self.states_explored = 0  # the number of states explored at any moment of time.
        
    
    def validate_input_matrix(self, matrix):
        ''' Validate the matrix. The matrix should have 0-8 values and no value should be repeated.
            While creating the puzzle object, validation should be done on start_state & goal_state '''
        max_row = len(matrix)
        max_column = len(matrix[0])
        
        for value in range(0, max_row*max_column):
            found = False;
            for i in range(max_row):
                for j in range(max_column):
                    if(value == matrix[i, j]):
                        found = True
            
            if(found == False):
                raise Exception("This is not a valid matrix")

        
    def get_neighbor_states(self, current_state):
        ''' Get the neighbors of the current state. The neighbors will be the states that comes
            after taking Left/Right/Up/Down move.
            No validation is done on the neighbor in this function.
            The called of this function need to do all the required validations'''
        
        move_directions = [("LEFT", 0, -1), ("RIGHT", 0, 1), ("UP", -1, 0), ("DOWN", 1, 0)]
        row, col = current_state.get_i_j_of(B)
        neighbors = []
        
        for move, i_offset, j_offset in move_directions:
            new_i = row + i_offset
            new_j = col + j_offset
            
            # move the blank space Left/Right/Up/Down if it is not crossing the boundries
            if ( 0 <= new_i < self.max_row ) and ( 0 <= new_j < self.max_column) :
                new_state = self.move(current_state, move, row[0], col[0])
                neighbors.append(new_state)

        return neighbors


    def move(self, current_state, move, i, j):
        '''MOve the blank space to Left/Right/Up/Down and return the new state
           The current state will be parent to tthis new state'''
        temp_matrix = current_state.get_copy()
        
        # Store the value of matrix[i, j] in temp varianble. This will be used during swap operation
        temp_val = temp_matrix[i, j]
        
        if move == "RIGHT":
            temp_matrix[i, j] = temp_matrix[i, j+1]
            temp_matrix[i, j+1] = temp_val
        elif move == "LEFT":
            temp_matrix[i, j] = temp_matrix[i, j-1]
            temp_matrix[i, j-1] = temp_val
        elif move == "UP":
            temp_matrix[i, j] = temp_matrix[i-1, j]
            temp_matrix[i-1, j] = temp_val
        elif move == "DOWN":
            temp_matrix[i, j] = temp_matrix[i+1, j]
            temp_matrix[i+1, j] = temp_val

        # Create the state from this new matrix. Assign the current matrix as parent to this new state
        new_state = node_state(temp_matrix, current_state, move, self.goal_state)
        return new_state

    def print_solution(self):
        ''' Print the solution '''

        print("\n---------------------")

        while len(self.solution_steps) != 0:
            current_state = self.solution_steps.pop(0)
            print("\nMove: ", current_state.move)
            current_state.print()
    
    def solve(self, open_list, heuristic_function):
        '''This fiunction can be used to find the goal state.
            It expect the instance of the list i.e. the open_list.
            This function will just call the add & remove function of the list it is
            responsiblity of the called to implement all dependent functions of the list'''
        
        # Keep looping until solution is not found
        while not open_list.empty():
            # Remove a state from the Queue/Stack
            current_state = open_list.remove()
            self.states_explored += 1
            
            # if hash of current state and Goal state is same then we have reached the goal :D
            if current_state.hash == self.goal_state.hash:
                self.end_time = datetime.now()

                item = current_state
                while item.parent is not None:
                    self.solution_steps.insert(0, item)
                    
                    self.number_of_states_to_optimal_path += 1
                    self.optimal_path_cost += item.actual_cost
                    self.total_of_fn += item.cost
                    
                    item = item.parent
                return True
            
            # get the valid neighbors of current state. Drop the neighbor that is already explored list i.e. close list
            neighbors = self.get_neighbor_states(current_state)
            for neighbor in neighbors:
                if neighbor.hash not in self.close_list:
                    self.close_list.append(neighbor.hash)
                    
                    neighbor.set_cost(heuristic_function)
                    open_list.add(neighbor)
        
        #Goal state is not found
        return False
    
    def display_statistics(self, type_of_algorithm, solution_found, heuristic_function):
        
        print(f"\n============ Result of A* search using heuristic{heuristic_function} ----------------")
        
        self.end_time = datetime.now()
        execution_time = self.end_time - self.start_time
        
        sol_found = " Yes" if solution_found == True else " No"
        print(f"Solution found: {sol_found}")
        
        print("Start state: ")
        self.start_state.print()
        
        print("\n Goal state: ")
        self.goal_state.print()
        
        
        print(f"Total number of states explored: {self.states_explored}")
        print(f"Time taken for execution : {execution_time}")
            
        if solution_found == True:
            print(f"Total number of states to the optimal path: {self.number_of_states_to_optimal_path}")
            print(f"Total cost g(n) =  {self.optimal_path_cost}")
            print(f"Total cost f(n) =  {self.total_of_fn}")
            print("Optimal Path is: ")
            self.print_solution()
            

In [5]:
class A_star_puzzle_solver(puzzle):
    
    def find_the_solution(self, heuristic_function):
        open_list = Priority_Queue()
        type_of_algorithm = "A* search"

        # Clear the explored list, solution steps
        self.close_list.clear()
        self.solution_steps.clear()

        self.start_state.set_cost(heuristic_function)
        open_list.add(self.start_state)
        self.close_list.append(self.start_state.hash)
        
        self.start_time = datetime.now()
        solution_found = self.solve(open_list, heuristic_function)
            
        self.display_statistics(type_of_algorithm, solution_found, heuristic_function)

In [6]:
B=0  # Just to make the matrix more readable as eyes will easily catch "B" compared to 0.

#take a random array
# random_array = np.arange(9)
# np.random.shuffle(random_array)

# enter the start state manually. (ToDo: If requirement is to generate random matrix, then need to write a funtion for that)
# start_state = random_array.reshape(3, 3)
start_state =np.array([
       [6, 7, 3],
       [8, 4, 2],
       [1, B, 5]])


# The goal state. This should be the goal where we need to reach
goal_state= np.array([
       [1, 2, 3],
       [4, 5, 6],
       [7, 8, B]])


h1 = A_star_puzzle_solver(start_state, goal_state)
h1.find_the_solution(1)

h2 = A_star_puzzle_solver(start_state, goal_state)
h2.find_the_solution(2)

h3 = A_star_puzzle_solver(start_state, goal_state)
h3.find_the_solution(3)

h4 = A_star_puzzle_solver(start_state, goal_state)
h4.find_the_solution(4)


============ Result of A* search using heuristic1 ----------------
Solution found:  No
Start state: 
6 7 3  
8 4 2  
1 B 5  

 Goal state: 
1 2 3  
4 5 6  
7 8 B  
Total number of states explored: 181440
Time taken for execution : 0:12:02.912468

============ Result of A* search using heuristic2 ----------------
Solution found:  No
Start state: 
6 7 3  
8 4 2  
1 B 5  

 Goal state: 
1 2 3  
4 5 6  
7 8 B  
Total number of states explored: 181440
Time taken for execution : 0:12:05.033437

============ Result of A* search using heuristic3 ----------------
Solution found:  No
Start state: 
6 7 3  
8 4 2  
1 B 5  

 Goal state: 
1 2 3  
4 5 6  
7 8 B  
Total number of states explored: 181440
Time taken for execution : 0:12:33.553659

============ Result of A* search using heuristic4 ----------------
Solution found:  No
Start state: 
6 7 3  
8 4 2  
1 B 5  

 Goal state: 
1 2 3  
4 5 6  
7 8 B  
Total number of states explored: 181440
Time taken for execution : 0:14:09.219398


## The given matrix can not be solved because the count of inversion is odd. Below is the code that can be used to find if any given matrix can be solved or not.

In [8]:
def is_matrix_solvable(matrix):

    inversions = 0
    
    arr2D = np.array(matrix)
    arr1D = arr2D.flatten()
    arr1D = np.delete(arr1D, (arr1D == 0))
    
    for i in range(len(arr1D)):
        for j in range(i + 1, len(arr1D)):
            if arr1D[i] > arr1D[j]:
                inversions += 1
                
    return (inversions%2 == 0)


print("Can matrix be solved? : ", is_matrix_solvable(start_state))


Can matrix be solved? :  False


In [9]:
another_start_state =np.array([
       [5, 3, 8],
       [1, 2, 4],
       [7, 6, B]])


h1 = A_star_puzzle_solver(another_start_state, goal_state)
h1.find_the_solution(1)

h2 = A_star_puzzle_solver(another_start_state, goal_state)
h2.find_the_solution(2)

h3 = A_star_puzzle_solver(another_start_state, goal_state)
h3.find_the_solution(3)

h4 = A_star_puzzle_solver(another_start_state, goal_state)
h4.find_the_solution(4)


============ Result of A* search using heuristic1 ----------------
Solution found:  Yes
Start state: 
5 3 8  
1 2 4  
7 6 B  

 Goal state: 
1 2 3  
4 5 6  
7 8 B  
Total number of states explored: 26243
Time taken for execution : 0:00:12.995637
Total number of states to the optimal path: 18
Total cost g(n) =  171
Total cost f(n) =  171
Optimal Path is: 

---------------------

Move:  UP
5 3 8  
1 2 B  
7 6 4  

Move:  UP
5 3 B  
1 2 8  
7 6 4  

Move:  LEFT
5 B 3  
1 2 8  
7 6 4  

Move:  DOWN
5 2 3  
1 B 8  
7 6 4  

Move:  RIGHT
5 2 3  
1 8 B  
7 6 4  

Move:  DOWN
5 2 3  
1 8 4  
7 6 B  

Move:  LEFT
5 2 3  
1 8 4  
7 B 6  

Move:  UP
5 2 3  
1 B 4  
7 8 6  

Move:  RIGHT
5 2 3  
1 4 B  
7 8 6  

Move:  UP
5 2 B  
1 4 3  
7 8 6  

Move:  LEFT
5 B 2  
1 4 3  
7 8 6  

Move:  LEFT
B 5 2  
1 4 3  
7 8 6  

Move:  DOWN
1 5 2  
B 4 3  
7 8 6  

Move:  RIGHT
1 5 2  
4 B 3  
7 8 6  

Move:  UP
1 B 2  
4 5 3  
7 8 6  

Move:  RIGHT
1 2 B  
4 5 3  
7 8 6  

Move:  DOWN
1 2 3  
4 5 B  
7 8 